<a href="https://colab.research.google.com/github/singhmansi25/Mansi-Singh-Spottabl/blob/main/Spottabl_MansiSingh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Engineer Assignment**

For this assignment, you’ll be provided with a dataset of companies as a list of JSON objects. Each object holds relevant data for a company. The fields are described below:

**tags**: the domains company operates in

**companyName**: synonyms of the company

**domainName**: extracted from the company website’s URL.

**companyDesc**: a short description of the company's operations

**fundingList**: details regarding the company’s funding rounds

**crunchbase_url**: URL to the company’s Crunchbase page

**employeeNumber**: number of employees in the company at the time data was extracted

**foundedDate**: date the company was founded on

**companyWebsite**: URL to company’s website

**domain_groups**: listing out possible groups the company may belong to. (this is our internal classification, may not be accurate)

**TYPE**: another internal classification (for product, and service-based companies), may not be accurate


#**Importing Libraries**

In [40]:
# Import useful libraries.
import json
import pandas as pd
from datetime import datetime
import datetime as dt
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

#**Reading Dataset**

In [41]:
# path of dataset
path = "/content/domainTags2.json"

# read json dataset 
json_dataframe = pd.read_json(path)

# **Feature Engineering**

In [42]:
# extracting ID from _id column
json_dataframe['oid'] = json_dataframe['_id'].apply(lambda x: x['$oid'])
first_column = json_dataframe.pop('oid')
# insert column using insert(position, column_name, first_column) function
json_dataframe.insert(0, 'Id', first_column)
# remove _id column
json_dataframe.pop('_id')

# Extract datetime in proper format from columns
json_dataframe['foundedDate'] = json_dataframe['foundedDate'].apply(lambda x: x if isinstance(x, float) else datetime.fromtimestamp(int(x['$date']['$numberLong']) / 1000) if isinstance(x, dict) and '$date' in x and '$numberLong' in x['$date'] else datetime.strptime(str(x['$date']), '%Y-%m-%dT%H:%M:%S.%fZ'))

json_dataframe['UpdatedOn'] = json_dataframe['UpdatedOn'].apply(lambda x: dt.datetime.strptime(x['$date'], '%Y-%m-%dT%H:%M:%S.%fZ') if isinstance(x, dict) and '$date' in x else None )

# replace missing values in UpdatedOn column with current datetime
datetime = dt.datetime.now()
current_date = datetime.today().strftime('%Y-%m-%dT%H:%M:%S.%fZ')
json_dataframe['UpdatedOn'].fillna(current_date, inplace=True)

# view json dataframe
json_dataframe.head()

,Id,tags,companyName,domainName,crunchbase_url,TYPE,UpdatedOn,companyDesc,companyWebsite,employeeNumber,foundedDate,domain_groups,fundingList
0,5d09dfd68501d21d7d729bb1,"[finance, insurance]",[21st century insurance],21st,https://www.crunchbase.com/organization/21st-c...,Product-based,2020-01-18 03:39:34.845000,"since 1958, 21st century insurance has been de...",www.21st.com,"{'max': 10000, 'min': 5001}",1958-01-01 09:00:00,"{'Fintech/ Payments': 0.825334256973911, 'Insu...",NaN
1,5d09dfd78501d21d7d729bbb,[3d technology],[3d networks],3dnetworks,https://www.crunchbase.com/organization/3d-net...,Product-based,2020-01-20 15:10:33.852000,NaN,www.3dnetworks.com,NaN,2000-01-01 09:00:00,NaN,NaN
2,5d09dfd78501d21d7d729bbc,"[3d technology, augmented reality, e-commerce]",[3dphy],3dphy,https://www.crunchbase.com/organization/3dphy-...,Product-based,2020-01-16 11:44:30.116000,3dphy provides a range of 3d virtual and augme...,www.3dphy.com,"{'max': 50, 'min': 11}",2014-11-03 09:00:00,NaN,[{'fundingDate': {'$date': '2014-12-18T09:00:0...
3,5d09dfd78501d21d7d729bbe,"[android, enterprise software, mobile]",[3lm],3lm,https://www.crunchbase.com/organization/3lm?ut...,Product-based,2020-01-21 12:50:14.767000,3lm has already signed deals to provide its te...,www.3lm.com,"{'max': 50, 'min': 11}",2010-07-01 09:00:00,{'Developer Tools': 0.617930901407716},[{'fundingDate': {'$date': '2010-07-01T09:00:0...
4,5d09dfd78501d21d7d729bbf,"[automotive, electronics, enterprise software]",[3m],3m,https://www.crunchbase.com/organization/3m?utm...,Product-based,2020-01-17 17:56:53.792000,3m operates as a diversified technology compan...,www.3m.com,"{'max': 10001, 'min': 0}",1902-01-01 09:00:00,"{'Hardware': 0.6879067738004571, 'Automotive':...",NaN


# **Missing Values**

In [44]:
json_dataframe.shape

(7462, 13)

In [45]:
json_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7462 entries, 0 to 7461
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Id              7462 non-null   object
 1   tags            7397 non-null   object
 2   companyName     7462 non-null   object
 3   domainName      7462 non-null   object
 4   crunchbase_url  7462 non-null   object
 5   TYPE            6097 non-null   object
 6   UpdatedOn       7462 non-null   object
 7   companyDesc     6247 non-null   object
 8   companyWebsite  7336 non-null   object
 9   employeeNumber  6604 non-null   object
 10  foundedDate     6603 non-null   object
 11  domain_groups   4406 non-null   object
 12  fundingList     2140 non-null   object
dtypes: object(13)
memory usage: 758.0+ KB


In [43]:
# Calculate number of missing values in dataframe
json_dataframe.isnull().sum()

Id                   0
tags                65
companyName          0
domainName           0
crunchbase_url       0
TYPE              1365
UpdatedOn            0
companyDesc       1215
companyWebsite     126
employeeNumber     858
foundedDate        859
domain_groups     3056
fundingList       5322
dtype: int64

# **Case-1**

You’ll be required to
given a company name (similar to ones present in companyName), return the close competitors of the company, using any of the fields (or combination of fields) from the dataset. For example, competitors of Swiggy would be Zomato, FoodPanda, DoorDash, etc. Since all of them operate in similar domains and offer similar services.

# **Approach**


1. Find the company's tags, domainName from the dataset based on the given company name.

2. Based on these fields to filter out companies from the dataset that operate in the same domains and offer similar services as the given company.

3. Rank the filtered companies based on their similarity to the given company. 

4. Return the top N companies with the highest similarity scores as the close competitors of the given company.



In [49]:
# create a copy of original dataframe
df = json_dataframe.copy()
df.head(10)

,Id,tags,companyName,domainName,crunchbase_url,TYPE,UpdatedOn,companyDesc,companyWebsite,employeeNumber,foundedDate,domain_groups,fundingList
0,5d09dfd68501d21d7d729bb1,"[finance, insurance]",[21st century insurance],21st,https://www.crunchbase.com/organization/21st-c...,Product-based,2020-01-18 03:39:34.845000,"since 1958, 21st century insurance has been de...",www.21st.com,"{'max': 10000, 'min': 5001}",1958-01-01 09:00:00,"{'Fintech/ Payments': 0.825334256973911, 'Insu...",NaN
1,5d09dfd78501d21d7d729bbb,[3d technology],[3d networks],3dnetworks,https://www.crunchbase.com/organization/3d-net...,Product-based,2020-01-20 15:10:33.852000,NaN,www.3dnetworks.com,NaN,2000-01-01 09:00:00,NaN,NaN
2,5d09dfd78501d21d7d729bbc,"[3d technology, augmented reality, e-commerce]",[3dphy],3dphy,https://www.crunchbase.com/organization/3dphy-...,Product-based,2020-01-16 11:44:30.116000,3dphy provides a range of 3d virtual and augme...,www.3dphy.com,"{'max': 50, 'min': 11}",2014-11-03 09:00:00,NaN,[{'fundingDate': {'$date': '2014-12-18T09:00:0...
3,5d09dfd78501d21d7d729bbe,"[android, enterprise software, mobile]",[3lm],3lm,https://www.crunchbase.com/organization/3lm?ut...,Product-based,2020-01-21 12:50:14.767000,3lm has already signed deals to provide its te...,www.3lm.com,"{'max': 50, 'min': 11}",2010-07-01 09:00:00,{'Developer Tools': 0.617930901407716},[{'fundingDate': {'$date': '2010-07-01T09:00:0...
4,5d09dfd78501d21d7d729bbf,"[automotive, electronics, enterprise software]",[3m],3m,https://www.crunchbase.com/organization/3m?utm...,Product-based,2020-01-17 17:56:53.792000,3m operates as a diversified technology compan...,www.3m.com,"{'max': 10001, 'min': 0}",1902-01-01 09:00:00,"{'Hardware': 0.6879067738004571, 'Automotive':...",NaN
5,5d09dfd78501d21d7d729bc0,"[enterprise software, gambling]",[500 startups],500,https://www.crunchbase.com/organization/500-co...,Product-based,2020-01-22 04:52:23.859000,500.com is a chinese online sports lottery ser...,www.500.com/,"{'max': 1000, 'min': 501}",2001-10-01 09:00:00,NaN,[{'fundingDate': {'$date': '2013-10-21T09:00:0...
6,5d09dfd78501d21d7d729bf0,"[health care, insurance, public relations]",[aci a verisk company],verisk,https://www.crunchbase.com/organization/verisk...,Product-based,2020-01-22 05:29:55.781000,"verisk analytics, inc. (verisk) enables risk-b...",www.verisk.com,"{'max': 5000, 'min': 1001}",1971-01-01 09:00:00,NaN,[{'fundingDate': {'$date': '2016-09-11T09:00:0...
7,5d09dfd78501d21d7d729c0c,"[electronics, manufacturing, product design]",[adcock ingram],adcock,https://www.crunchbase.com/organization/adcock...,Product-based,2020-01-19 19:59:56.775000,NaN,www.adcock.co.uk/,"{'max': 1000, 'min': 501}",1964-01-01 09:00:00,{'Hardware': 0.785256187259849},NaN
8,5d09dfd88501d21d7d729c23,"[industrial, security, software]","[tyco fire security india, tyco international,...",tyco,https://www.crunchbase.com/organization/tyco-i...,Product-based,2020-01-21 06:44:14.399000,tyco international (nyse: tyc) is a diversifie...,www.tyco.com,"{'max': 10001, 'min': 0}",1960-01-01 09:00:00,NaN,NaN
9,5d09dfd88501d21d7d729c39,"[analytics, artificial intelligence, automotiv...",[affectiva],affectiva,https://www.crunchbase.com/organization/affect...,Product-based,2020-01-21 01:27:24.378000,"affectiva, an mit media lab spin-off, is a pio...",www.affectiva.com,"{'max': 100, 'min': 51}",2009-01-01 09:00:00,NaN,NaN


In [50]:
## removing unwanted columns from dataframe
cols_to_remove = ['companyName', 'crunchbase_url', 'UpdatedOn', 'companyDesc', 'companyWebsite', 'employeeNumber', 'foundedDate', 'fundingList']
df = df.drop(columns=cols_to_remove)

In [51]:
df.head(10)

,Id,tags,domainName,TYPE,domain_groups
0,5d09dfd68501d21d7d729bb1,"[finance, insurance]",21st,Product-based,"{'Fintech/ Payments': 0.825334256973911, 'Insu..."
1,5d09dfd78501d21d7d729bbb,[3d technology],3dnetworks,Product-based,NaN
2,5d09dfd78501d21d7d729bbc,"[3d technology, augmented reality, e-commerce]",3dphy,Product-based,NaN
3,5d09dfd78501d21d7d729bbe,"[android, enterprise software, mobile]",3lm,Product-based,{'Developer Tools': 0.617930901407716}
4,5d09dfd78501d21d7d729bbf,"[automotive, electronics, enterprise software]",3m,Product-based,"{'Hardware': 0.6879067738004571, 'Automotive':..."
5,5d09dfd78501d21d7d729bc0,"[enterprise software, gambling]",500,Product-based,NaN
6,5d09dfd78501d21d7d729bf0,"[health care, insurance, public relations]",verisk,Product-based,NaN
7,5d09dfd78501d21d7d729c0c,"[electronics, manufacturing, product design]",adcock,Product-based,{'Hardware': 0.785256187259849}
8,5d09dfd88501d21d7d729c23,"[industrial, security, software]",tyco,Product-based,NaN
9,5d09dfd88501d21d7d729c39,"[analytics, artificial intelligence, automotiv...",affectiva,Product-based,NaN


In [52]:
# function to find related companies for a given company based on domain score

def get_close_competitors(company_name, dataset, n=10):
    # find companies that match the given name
    matches = []
    for cmp in dataset:
        if (cmp.get('tags') and company_name in cmp['tags']) or \
           (cmp.get('domainName') and company_name in cmp['domainName']):
            matches.append(cmp)
    
    if not matches:
        return []
    
    # extract tags and company names of the matching companies
    tags = set()
    names = set()
    for cmp in matches:
        if cmp.get('tags'):
            tags.update(cmp['tags'])
        if cmp.get('domainName'):
            names.update(cmp['domainName'])
    
    # find close competitors based on tags and company names
    competitors = []
    for cmp in dataset:
        if cmp in matches:
            continue
        score = 0
        if cmp.get('tags'):
            score += len(set(cmp['tags']).intersection(tags))
        if cmp.get('domainName'):
            score += len(set(cmp['domainName']).intersection(names))
        if score > 0:
            competitors.append({'companyName': cmp.get('companyName', ''),
                                'score': score})
    
    # sort competitors by score and return the top N
    competitors.sort(key=lambda x: x['score'], reverse=True)
    return competitors[:n]


In [54]:
# test the function for a given company name
company_name = 'amdocs'
with open(path, 'r') as f:
    dataset = json.load(f)

competitors = get_close_competitors(company_name, dataset)
for competitor in competitors:
    print('Company Name:  ',competitor['companyName'])

Company Name:   ['indium software']
Company Name:   ['traction on demand']
Company Name:   ['agile crm']
Company Name:   ['activevideo']
Company Name:   ['adreno technologies']
Company Name:   ['cdc software']
Company Name:   ['cornerstone ondemand']
Company Name:   ['crayon data']
Company Name:   ['codemancers']
Company Name:   ['cloudagents inc comforce zrt']


# **Case-2**

Given a domain (any one of the domains in ‘tags’ field), present the companies with the fastest, and the slowest growth (hint: it may be based on the fundingList, foundedDate, etc)

# **Approach**

1. Filter out the companies from the dataset that operate in the given domain.

2. Calculate the growth rate for each company based on their founded year.

3. Rank the companies based on their year difference i.e current year and founded year.

4. Return the company with the highest growth rate as the fastest growing company and the company with the lowest growth rate as the slowest growing company.

In [55]:
# extract founded year from founded date
json_dataframe['foundedYr'] = pd.to_datetime(json_dataframe['foundedDate'], errors = 'coerce').fillna(pd.to_datetime('2000-01-01'))
json_dataframe['foundedYr'] = json_dataframe['foundedYr'].dt.year
json_dataframe['foundedYr'] = json_dataframe['foundedYr'].astype(int)

In [56]:
json_dataframe.head()

,Id,tags,companyName,domainName,crunchbase_url,TYPE,UpdatedOn,companyDesc,companyWebsite,employeeNumber,foundedDate,domain_groups,fundingList,foundedYr
0,5d09dfd68501d21d7d729bb1,"[finance, insurance]",[21st century insurance],21st,https://www.crunchbase.com/organization/21st-c...,Product-based,2020-01-18 03:39:34.845000,"since 1958, 21st century insurance has been de...",www.21st.com,"{'max': 10000, 'min': 5001}",1958-01-01 09:00:00,"{'Fintech/ Payments': 0.825334256973911, 'Insu...",NaN,1958
1,5d09dfd78501d21d7d729bbb,[3d technology],[3d networks],3dnetworks,https://www.crunchbase.com/organization/3d-net...,Product-based,2020-01-20 15:10:33.852000,NaN,www.3dnetworks.com,NaN,2000-01-01 09:00:00,NaN,NaN,2000
2,5d09dfd78501d21d7d729bbc,"[3d technology, augmented reality, e-commerce]",[3dphy],3dphy,https://www.crunchbase.com/organization/3dphy-...,Product-based,2020-01-16 11:44:30.116000,3dphy provides a range of 3d virtual and augme...,www.3dphy.com,"{'max': 50, 'min': 11}",2014-11-03 09:00:00,NaN,[{'fundingDate': {'$date': '2014-12-18T09:00:0...,2014
3,5d09dfd78501d21d7d729bbe,"[android, enterprise software, mobile]",[3lm],3lm,https://www.crunchbase.com/organization/3lm?ut...,Product-based,2020-01-21 12:50:14.767000,3lm has already signed deals to provide its te...,www.3lm.com,"{'max': 50, 'min': 11}",2010-07-01 09:00:00,{'Developer Tools': 0.617930901407716},[{'fundingDate': {'$date': '2010-07-01T09:00:0...,2010
4,5d09dfd78501d21d7d729bbf,"[automotive, electronics, enterprise software]",[3m],3m,https://www.crunchbase.com/organization/3m?utm...,Product-based,2020-01-17 17:56:53.792000,3m operates as a diversified technology compan...,www.3m.com,"{'max': 10001, 'min': 0}",1902-01-01 09:00:00,"{'Hardware': 0.6879067738004571, 'Automotive':...",NaN,1902


In [57]:
# function to find growth of company and return fastest growing and slowest growing company based on years as there are lost of missing values based on funds

def growth_of_company(df, domain):
  # Filter the dataframe to include only the companies in the given domain
  domain_df = df[df['tags'].apply(lambda x: domain in x if isinstance(x, list) else False)]

  # Calculate the age of each company
  current_year = pd.to_datetime('now').year
  domain_df['years_since_founded'] = current_year - domain_df['foundedYr'] 

  # # Sort the companies by year in descending order
  domain_df = domain_df.sort_values(by='years_since_founded', ascending=False)

  # Print the companies with the fastest and slowest growth
  print('Fastest Growing Companies in the {} Domain:'.format(domain))
  print(domain_df.head(10)['domainName'])
  print('\n\n')
  print('Slowest Growing Companies in the {} Domain:'.format(domain))
  print(domain_df.tail(10)['domainName'])



In [59]:
# Load the dataset into a pandas dataframe
df = json_dataframe.copy()

# Define the domain of interest
domain = 'insurance'

# call function growth_of_company
growth_of_company(df, domain)

Fastest Growing Companies in the insurance Domain:
443                    axa
6576                   dnb
5348    willistowerswatson
2968       legalandgeneral
5026                  unum
5344            wellsfargo
3029    northwesternmutual
7131          guardianlife
4558               swissre
3245               metlife
Name: domainName, dtype: object



Slowest Growing Companies in the insurance Domain:
4851       turtlemint
4205        singsaver
7317           google
6474          godigit
6206    useprotection
6655           reniso
6104          nfaktor
724              atma
57               acko
1610              dxc
Name: domainName, dtype: object


<ipython-input-57-a3e63e1e3d80>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  domain_df['years_since_founded'] = current_year - domain_df['foundedYr']
